In [1]:
import pandas as pd
from textblob import TextBlob

In [2]:
topics_train = pd.read_csv('/home/juan/Documents/Academico/Maestria/Text Minnig/Proyecto/text_mining/individual_versions/Ed/Output_files/train.csv')
topics_test = pd.read_csv('/home/juan/Documents/Academico/Maestria/Text Minnig/Proyecto/text_mining/individual_versions/Ed/Output_files/test.csv')
individual_sum = pd.read_csv('/home/juan/Documents/Academico/Maestria/Text Minnig/Proyecto/text_mining/Data/embeddings_sum_individual.zip')
individual_averages = pd.read_csv('/home/juan/Documents/Academico/Maestria/Text Minnig/Proyecto/text_mining/Data/embeddings_average_individual.zip')

In [3]:
topics_train.columns = list(topics_test)

In [17]:
topics = [topics_train, topics_test]
topics_full = pd.concat(topics, ignore_index=True)

In [5]:
RedditNews = pd.read_csv('/home/juan/Documents/Academico/Maestria/Text Minnig/Proyecto/text_mining/Data/RedditNews.csv')
combined = pd.read_csv('/home/juan/Documents/Academico/Maestria/Text Minnig/Proyecto/text_mining/Data/Combined_News_DJIA.csv')

In [6]:
RedditNews = RedditNews[RedditNews['Date'].isin(combined['Date'])]

In [7]:
## Hay error en la codificación de caracteres especiales, encontré ese, pero hay que ver que otros surgen
RedditNews['News'] = RedditNews['News'].str.replace("year.old", " year old ", regex=True)
RedditNews['News'] = RedditNews['News'].str.replace("\d year", " year ", regex=True)
RedditNews['News'] = RedditNews['News'].str.replace("year old", "years old", regex=True)


RedditNews['News'] = RedditNews['News'].str.replace("years.old", " years old ", regex=True)
RedditNews['News'] = RedditNews['News'].str.replace("\d years", " years ", regex=True)
## Hay error en la codificación de caracteres especiales, encontré ese, pero hay que ver que otros surgen
index_review = RedditNews[(RedditNews['News'].str.startswith('b"')) |
                         (RedditNews['News'].str.startswith("b'"))].index

display(RedditNews[RedditNews['News'].str.startswith('b"')].head(),
        RedditNews[RedditNews['News'].str.startswith("b'")].head())


RedditNews['News'] = RedditNews['News'].str.replace('^b\"', " ", regex=True)
RedditNews['News'] = RedditNews['News'].str.replace("^b\'", " ", regex=True)
import re
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stoplist = stopwords.words("english")
REPLACE_BY_SPACE_RE = re.compile('[(){}\[\]\|@,;-]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z ^#\+_]')
SEP_NUMBER = re.compile('(?<=\d)\,|\.(?=\d)')
USA_ABREV = re.compile('U\.S|u\.s\|u\.s\.a\.|US')
DOT_ABREV = re.compile('\.(?![a-zA-Z]{2})')
STARTING_B = re.compile("^\"b' |^b ")
STOPWORDS = stopwords.words('english')

def clean_text(text):
    """
        text: a string
        return: modified initial string
    """
    
    text = USA_ABREV.sub(' usa ', text) # replace U.S U.S. u.s US for usa
    text = text.lower() # lowercase text
    text = text.replace("al-qaeda", "alqaeda")
    text = text.replace("al-qa'eda", "alqaeda")
    text = text.replace('&amp;', '&')
    text = text.replace('&', '')    

    text = DOT_ABREV.sub('', text) # removes abrevetion dot, ej: L.G.B.T  = LGBT
    text = SEP_NUMBER.sub('', text) # removes . and , seprating numbers
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub(' ', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = STARTING_B.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text

    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    text = text.strip()
    return text
    
RedditNews['News'] = RedditNews['News'].apply(clean_text)

,Date,News
54798,2010-06-30,"b""South Korea's parliament votes to legalize c..."
54804,2010-06-30,"b""The German economy is rapidly improving, wit..."
54819,2010-06-30,"b""BBC News - Russian spy suspect missing in Cy..."
54821,2010-06-30,"b""Iraq inquiry: secret documents showing Tony ..."
54822,2010-06-30,"b""Apartheid loves apartheid: Israel's secret r..."


,Date,News
54799,2010-06-30,b'Pope rebukes cardinal who exposed abuse: \nP...
54800,2010-06-30,b'This depression is similar to the Great Pani...
54801,2010-06-30,b'The Niger Delta has experienced oil spills o...
54802,2010-06-30,b'G20 Toronto - So Black Block get green light...
54803,2010-06-30,b'Half of Afghanistans 476 women prisoners wer...


In [8]:
mask = RedditNews['News'].duplicated(keep=False)
print(len(RedditNews[mask]))
RedditNews[mask].sort_values('News')

131


,Date,News
15303,2014-10-27,4 banks including jpmorgan fined europe cartel...
15299,2014-10-28,4 banks including jpmorgan fined europe cartel...
20850,2014-03-19,900 workers already died building qatars world...
20998,2014-03-14,900 workers already died building qatars world...
27847,2013-06-13,afghan parliament upholds right marry children
...,...,...
20311,2014-04-10,usa troops may sent eastern europe
1652,2016-04-26,watchdog says press freedom decline new era pr...
1780,2016-04-21,watchdog says press freedom decline new era pr...
50278,2010-12-28,wikileaks swedish government hid anti terror o...


In [8]:
algo = RedditNews.reset_index(drop = True)

In [9]:
algo.drop_duplicates(subset=['News'], keep='first', inplace=True)

In [10]:
lista = list(algo.index)

In [11]:
sum_filt = individual_sum.iloc[lista,].drop(columns=['Unnamed: 0'])
average_filt = individual_averages.iloc[lista,].drop(columns=['Unnamed: 0'])

In [12]:
names = {}
for i in range(10):
    names[str(i)] = 'topic_' + str(i)

In [18]:
topics_full = topics_full.rename(columns = names)

In [19]:
sentiment = []
for i in algo.News:
    sentiment.append(TextBlob(i).sentiment.polarity)

In [20]:
topics_only = topics_full[list(names.values())+ ['Top', 'Date']]

In [21]:
sums_topics = pd.merge(sum_filt, topics_only, left_on=['Date','Top'] , right_on=['Date','Top'] , how='left')
averages_topics = pd.merge(average_filt, topics_only, left_on=['Date','Top'] , right_on=['Date','Top'] , how='left')

In [25]:
sums_topics.to_csv('sum_topics.csv', index=False)
averages_topics.to_csv('averages_topics.csv', index=False)

In [29]:
sum_filt['sentiment'] = sentiment
average_filt['sentiment'] = sentiment
sums_topics['sentiment'] = sentiment
averages_topics['sentiment'] = sentiment
sum_filt.to_csv('sum_sentiment.csv', index=False)
average_filt.to_csv('averages_sentiment.csv', index=False)
sums_topics.to_csv('sum_topics_sentiment.csv', index=False)
averages_topics.to_csv('averages_topics_sentiment.csv', index=False)

In [37]:
topics_sentiment = averages_topics[['Top', 'Date' , 'sentiment' , 'Label'] + list(names.values()) ]
topics_sentiment.to_csv('topics_sentiment.csv', index=False)

In [32]:
list(averages_topics)

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138'

In [15]:
only_topics = topics_full[list(names.values())]

In [16]:
sum_filt = sum_filt.reset_index(drop = True)
only_topics = only_topics.reset_index(drop = True)

In [22]:
full_sum = pd.concat([sum_filt, only_topics] , axis = 1)

In [23]:
average_filt = average_filt.reset_index(drop = True)
full_avg = pd.concat([average_filt, only_topics] , axis = 1)

In [27]:
full_avg.to_csv('averages_topics.csv', index=False)

In [28]:
full_sum.to_csv('sum_topics.csv', index=False)

In [10]:
algo['sentiment'] = sentiment

In [44]:
algo['Date'] =  pd.to_datetime(algo['Date'], format='%Y-%m-%d')
algo.sort_values('Date', inplace=True)

In [19]:
averages = pd.read_csv('/home/juan/Documents/Academico/Maestria/Text Minnig/Proyecto/text_mining/Data/averages_topics.zip')
sums = pd.read_csv('/home/juan/Documents/Academico/Maestria/Text Minnig/Proyecto/text_mining/Data/sum_topics.zip')

In [24]:
averages_solo = averages.drop(list(names.values()), axis = 1)
sums_solo = sums.drop(list(names.values()), axis = 1)

In [28]:
sums_solo['sentiment'] = sentiment
averages_solo['sentiment'] = sentiment
sums['sentiment'] = sentiment
averages['sentiment'] = sentiment

In [36]:
sums_solo.to_csv('sum_sentiment.csv', index=False)
averages_solo.to_csv('averages_sentiment.csv', index=False)
sums.to_csv('sum_topics_sentiment.csv', index=False)
averages.to_csv('averages_topics_sentiment.csv', index=False)

In [48]:
algo

,Date,News
0,2016-07-01,117 years old woman mexico city finally receiv...
1,2016-07-01,imf chief backs athens permanent olympic host
2,2016-07-01,president france says brexit donald trump
3,2016-07-01,british man must give police 24 hours notice s...
4,2016-07-01,100+ nobel laureates urge greenpeace stop oppo...
...,...,...
49713,2008-08-08,pentagon thinks attacking iran bad idea usa ne...
49714,2008-08-08,caucasus crisis georgia invades south ossetia
49715,2008-08-08,indian shoe manufactory series like work
49716,2008-08-08,visitors suffering mental illnesses banned oly...


In [42]:
import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = algo.News.values.tolist()
data_words = list(sent_to_words(data))

In [43]:
print(data_words[:1][0][:30])

['years', 'old', 'woman', 'mexico', 'city', 'finally', 'received', 'birth', 'certificate', 'died', 'hours', 'later', 'trinidad', 'alvarez', 'lira', 'waited', 'years', 'proof', 'born']
